In [0]:
import inspect
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation
from tensorflow.python.keras.layers import Add
from tensorflow.python.keras.layers import AveragePooling2D
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import GlobalAveragePooling2D
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Reshape
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
from keras.models import load_model
from sklearn.model_selection import train_test_split
# import pydotplus
from IPython.display import Image
# preprocessing
import math
import numpy as np
import matplotlib.pyplot as plt


In [0]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
!df -h

In [0]:
# google-drive-ocamlfuseのインストール
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Colab用のAuth token作成
from google.colab import auth
auth.authenticate_user()

# Drive FUSE library用のcredential生成
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# drive/ を作り、そこにGoogle Driveをマウントする
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
def check_images(side, left, right):
    graph_size = 7

    fig, axes = plt.subplots(side, side * 2,
                             figsize=(graph_size * 2, graph_size))
    for (row, col), ax in np.ndenumerate(axes):
        if col < side:
            ax.imshow(left[row * side + col])
        else:
            ax.imshow(right[row * side + col - side])
        plt.setp(ax, xticks=(), yticks=())
    plt.show()

In [0]:
cd /content/drive

In [0]:
NUM_CLASS = 17 #クラスの数
IMG_SIZE = 224 #画像のサイズ

img_dir = "/content/drive/data/img_weather"
X = []
Y = []

for i in range(NUM_CLASS):
    dir = "{0}/{1}".format(img_dir, i)
    for picture in list_pictures(dir):
        img = img_to_array(load_img(picture, target_size=(IMG_SIZE,IMG_SIZE)))
        X.append(img)
        Y.append(i)

X = np.asarray(X)
Y = np.asarray(Y)

print("X: ", X.shape)
print("Y: ", Y)

X = X.astype(np.float32) / 255.0
#one_hotベクトルに変換
Y = to_categorical(Y, NUM_CLASS)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


#学習用の画像とラベル
print(X_train.shape)
print(y_train.shape)

#テスト用の画像とラベル
print(X_test.shape)
print(y_test.shape)

In [0]:
train_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                               height_shift_range=0.1, shear_range=0.05,
                               zoom_range=0.1, horizontal_flip=True)

test_gen = ImageDataGenerator()

In [0]:
# check用
side=4
train_batch = train_gen.flow(X_train, y_train, batch_size=side ** 2,
                             shuffle=False)
test_batch = test_gen.flow(X_test, y_test, batch_size=side ** 2, shuffle=False)

train, _ = next(train_batch)
test, _ = next(test_batch)

check_images(side, train, test)

In [0]:
batch_size=128
# データを水増ししながら学習
train_iter = train_gen.flow(X_train, y_train, batch_size=batch_size, shuffle=True)

test_iter = test_gen.flow(X_test, y_test, batch_size=batch_size, shuffle=False)

steps_per_epoch=math.ceil(y_train.size / batch_size)
validation_steps=math.ceil(y_test.size / batch_size)

print(steps_per_epoch)
print(validation_steps)

In [0]:

# 学習済みモデルの読み込み
K.clear_session()
num_classes = 17

vgg = VGG16(weights='imagenet', include_top=True)
w = vgg.layers[-2].get_weights()

model = Sequential(vgg.layers[:-1])
for l in model.layers:
    l.trainable = False

model.add(Dense(num_classes, activation='softmax'))
model.layers[-2].trainable = True

# ckpt = ModelCheckpoint(filepath='/content/drive/models/ckpt{epoch:02d}-{val_loss:.2f}.test')
sgd = SGD(lr=1e-4, momentum=0.9)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# saveしたモデルの続きから
model_dir = "/content/drive/models/my_model.h5"
model = load_model(model_dir)

In [0]:
h = model.fit_generator(
    train_iter , 
    steps_per_epoch=steps_per_epoch,
    epochs=20, 
    validation_data=test_iter ,
    validation_steps=validation_steps,
#    callbacks=[ckpt], 
    verbose=1)

# モデルの保存
model.save('/content/drive/models/my_model.h5')

print('train loss:      ',h.history['loss'])
print('validation loss: ',h.history['val_loss'])

print('train accuracy:      ',h.history['acc'])
print('validation accuracy: ',h.history['val_acc'])

In [0]:
print('train loss:      ',h.history['loss'])
print('validation loss: ',h.history['val_loss'])

print('train accuracy:      ',h.history['acc'])
print('validation accuracy: ',h.history['val_acc'])

epochs=20
epoch = np.arange(epochs) + 1

plt.title('Loss')
plt.plot(epoch, h.history['loss'], label='train')
plt.plot(epoch, h.history['val_loss'], label='validation')
plt.legend()
plt.show()
plt.title('Accuracy')
plt.plot(epoch, h.history['acc'], label='train')
plt.plot(epoch, h.history['val_acc'], label='validation')
plt.legend()

In [0]:
import os.path

NUM_CLASS = 17 #クラスの数
IMG_SIZE = 224 #画像のサイズ

img_dir     = "/content/drive/data/img_weather"
X = []
fileid = []

for picture in list_pictures(img_dir):
  img = img_to_array(load_img(picture, target_size=(IMG_SIZE,IMG_SIZE)))
  X.append(img)
  id, ext = os.path.splitext(os.path.basename(picture))
  fileid.append(id)
  
X = np.asarray(X)
X = X.astype(np.float32) / 255.0

result = model.predict(X)

In [0]:
for i, id in enumerate(fileid):
  result = np.array(result)
  print("{0},{1}".format(id, result[i].argmax()))